In [2]:
## crawler
from bs4 import BeautifulSoup
from urllib import parse
import requests
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

## database save
import pandas as pd
import sqlite3

In [5]:
def get_article_info(main_url = "https://www.yesasia.ru/cat/c-news"):
    
    ## getting html source
    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    driver = webdriver.Chrome(options=options)
    driver.get(main_url)

    ## scroll down to the bottom
    for i in range(10):
        ActionChains(driver).key_down(Keys.END).perform()
        time.sleep(3)
        ActionChains(driver).key_up(Keys.END).perform()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    html_source = driver.page_source
    driver.close()

    ## html source parsing
    soup = BeautifulSoup(html_source, "html.parser")
    article_tag_list = soup.select('h3.entry-title.td-module-title > a')
    
    article_links = [a['href'] for a in article_tag_list]
    article_ids = [int(i.split('/')[-1])for i in article_links]

    return article_links, article_ids

def get_article_contents(article_links):
    
    article_title_list = []
    article_body_list = []
    
    for link in article_links:
        html_source = requests.get(link)
        soup = BeautifulSoup(html_source.text, "html.parser")

        article_title = soup.select('h1.entry-title')[0].text

        article_body = soup.select('div.td-post-content.tagdiv-type p')
        article_body = [b.text for b in article_body]
        article_body = [b for b in article_body if b.find('Смотрите также:') == -1]

        article_title_list.append(article_title)
        article_body_list.append((article_title,article_body))

    return article_body_list

### crawl

In [ ]:
article_links, article_ids = get_article_info()
article_body_list = get_article_contents(article_links)

In [7]:
## data migration to rueng.sqlite3

import pandas as pd

yesasia_articletitle = pd.read_csv(r'yesasia_articletitle.csv', encoding='utf-8')
yesasia_articlebody = pd.read_csv(r'yesasia_articlebody.csv', encoding='utf-8')

In [11]:
yesasia_articletitle.head()

,article_id,title,is_read,update_date
0,147897,Сколько денег Ли Су Ман и Ян Хен Сок потеряли ...,0,2023-11-14 13:18:58.166199
1,170834,На ком костюмчик лучше сидит? Хёмин vs Хёна,0,2023-11-14 13:18:59.432751
2,201870,"Ким Су Хён, Ким Джон Кук и Кристал выбраны сам...",0,2023-11-14 13:18:58.544118
3,213962,"13 моментов, когда B.A.P показывали любовь к BABY",0,2023-11-14 13:18:58.355574
4,240781,Выпускник университета Конкук изнасиловал студ...,0,2023-11-14 13:18:58.818949


In [12]:
yesasia_articlebody.head()

,contents_id,contents,update_date,article_id
0,1,Недавно было объявлено о съёмках тайваньского ...,2023-11-14 13:18:22.055519,1341586
1,2,СМОТРИТЕ ТАКЖЕ: Бай Лу и Чжан Лин Хэ понадобил...,2023-11-14 13:18:22.065492,1341586
2,3,Главную мужскую роль исполнит популярный тайва...,2023-11-14 13:18:22.072471,1341586
3,4,Эрик Чоу в образе главного героя,2023-11-14 13:18:22.079452,1341586
4,5,Главную женскую роль исполнит гонконгская актр...,2023-11-14 13:18:22.087451,1341586


In [13]:
## db save

conn = sqlite3.connect('rueng.sqlite3')
cursor = conn.cursor()

## articletitle
title_tuples = [(id_, title, is_read, update_date) for id_, title, is_read, update_date  in zip(yesasia_articletitle.article_id, yesasia_articletitle.title, yesasia_articletitle.is_read, yesasia_articletitle.update_date)]
cursor.executemany("INSERT INTO yesasia_articletitle VALUES (?, ?, ?, ?)", title_tuples)

## articlebody
body_tuples = [(id_, content, update_date, article_id) for id_, content, update_date, article_id in zip(yesasia_articlebody.contents_id, yesasia_articlebody.contents, yesasia_articlebody.update_date, yesasia_articlebody.article_id)]
cursor.executemany("INSERT INTO yesasia_articlebody VALUES (?, ?, ?, ?)", body_tuples)

conn.commit()
conn.close()